In [1]:
import numpy as np
import copy
import random
import pandas as pd
from community_detection import *


/Users/alex_1/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:

btc_wbtc = pd.read_csv("binance_data/Wrapped BTC/2024/1m/merged_closing_prices.csv", index_col=0, parse_dates=True)

eth_wbeth = pd.read_csv('binance_data/ETH_and_WBETH/2024/1m/merged_closing_prices.csv', index_col=0, parse_dates=True)

top_100 = pd.read_csv('binance_data/top_100_tickers/2024/1m/merged_closing_prices.csv', index_col=0, parse_dates=True)

btc_wbtc.drop(columns=['BTC/USDT_2024_1m'], inplace=True)
eth_wbeth.drop(columns=['ETH/USDT_2024_1m'], inplace=True)

#Merge the two dataframes on the index
wbtc_weth = btc_wbtc.merge(eth_wbeth, how='inner', left_index=True, right_index=True)

# #Merge the final dataframes on the index
merged_df = pd.merge(top_100, wbtc_weth, how='inner', left_index=True, right_index=True)


In [21]:
#Compute log returns
merged_df_log_returns = np.log(merged_df / merged_df.shift(1)).dropna()

#Compute standardized log returns
merged_df_standardized_log_returns = (merged_df_log_returns - merged_df_log_returns.mean()) / merged_df_log_returns.std()



In [23]:
merged_df_standardized_log_returns.columns

Index(['SHIB/USDT_2024_1m', 'BCH/USDT_2024_1m', 'DOGE/USDT_2024_1m',
       'ETC/USDT_2024_1m', 'MANA/USDT_2024_1m', 'GALA/USDT_2024_1m',
       'WLD/USDT_2024_1m', 'UNI/USDT_2024_1m', 'GRT/USDT_2024_1m',
       'IMX/USDT_2024_1m', 'QNT/USDT_2024_1m', 'JTO/USDT_2024_1m',
       'AXS/USDT_2024_1m', 'EGLD/USDT_2024_1m', 'SAND/USDT_2024_1m',
       'HBAR/USDT_2024_1m', 'PAXG/USDT_2024_1m', 'FLOW/USDT_2024_1m',
       'SEI/USDT_2024_1m', 'MKR/USDT_2024_1m', 'TRX/USDT_2024_1m',
       'BNB/USDT_2024_1m', 'BTC/USDT_2024_1m', 'NEAR/USDT_2024_1m',
       'ICP/USDT_2024_1m', 'SUI/USDT_2024_1m', 'IOTA/USDT_2024_1m',
       'PEPE/USDT_2024_1m', 'VET/USDT_2024_1m', 'LDO/USDT_2024_1m',
       'NEO/USDT_2024_1m', 'ETH/USDT_2024_1m', 'OM/USDT_2024_1m',
       'BONK/USDT_2024_1m', 'DEXE/USDT_2024_1m', 'AAVE/USDT_2024_1m',
       'FDUSD/USDT_2024_1m', 'XRP/USDT_2024_1m', 'AVAX/USDT_2024_1m',
       'ENS/USDT_2024_1m', 'STX/USDT_2024_1m', 'JASMY/USDT_2024_1m',
       'OP/USDT_2024_1m', 'ATOM/USDT_2024_1

In [24]:

correlation_matrix,T,N,company_names = create_correlation_matrix(merged_df_standardized_log_returns) 
C_g = calculate_C_g(correlation_matrix,T,N) #Modularity matrix
print(correlation_matrix.shape)
correlation_matrix_np = np.array(correlation_matrix)
louvain_communities = modified_louvain(C_g,correlation_matrix_np)

print("Global modularity of modified Louvain method: ", calculate_global_modularity(louvain_communities, C_g, correlation_matrix_np))


(66, 66)
Detected 2 Initially
Final number of communities: 2
Global modularity of modified Louvain method:  0.009285536037228996


In [25]:
louvain_company_communities = map_communities_to_company_names(louvain_communities,company_names)

In [30]:

print("LOUVAIN COMPANY COMMUNITIES")
print("---------------------------")
for louvain_company_community in louvain_company_communities:
    print(louvain_company_community)

LOUVAIN COMPANY COMMUNITIES
---------------------------
['WLD/USDT_2024_1m', 'ICP/USDT_2024_1m', 'HBAR/USDT_2024_1m', 'INJ/USDT_2024_1m', 'IMX/USDT_2024_1m', 'FLOW/USDT_2024_1m', 'OP/USDT_2024_1m', 'FLOKI/USDT_2024_1m', 'SEI/USDT_2024_1m', 'IOTA/USDT_2024_1m', 'ARB/USDT_2024_1m', 'ATOM/USDT_2024_1m', 'ENS/USDT_2024_1m', 'AVAX/USDT_2024_1m', 'JASMY/USDT_2024_1m', 'GALA/USDT_2024_1m', 'JTO/USDT_2024_1m', 'VET/USDT_2024_1m', 'LINK/USDT_2024_1m', 'FET/USDT_2024_1m', 'EOS/USDT_2024_1m', 'SAND/USDT_2024_1m', 'FIL/USDT_2024_1m', 'CRV/USDT_2024_1m', 'DOT/USDT_2024_1m', 'AXS/USDT_2024_1m', 'EGLD/USDT_2024_1m', 'MANA/USDT_2024_1m', 'USDC/USDT_2024_1m', 'THETA/USDT_2024_1m', 'NEAR/USDT_2024_1m', 'LDO/USDT_2024_1m', 'OM/USDT_2024_1m', 'GRT/USDT_2024_1m', 'BONK/USDT_2024_1m', 'SUI/USDT_2024_1m', 'APT/USDT_2024_1m', 'TIA/USDT_2024_1m', 'NEO/USDT_2024_1m', 'ALGO/USDT_2024_1m', 'UNI/USDT_2024_1m']
['WBTC/USDT_2024_1m', 'WBETH/USDT_2024_1m', 'PAXG/USDT_2024_1m', 'SOL/USDT_2024_1m', 'DEXE/USDT_2024_1m',

# Round 2 of community detection 
---

In [28]:
#Create filtered dataframes

filtered_dfs = []
for i, louvain_company_community in enumerate(louvain_company_communities):
    filtered_df = merged_df_standardized_log_returns[louvain_company_community]
    filtered_dfs.append(filtered_df)
    print(f"Filtered DataFrame {i+1} shape: {filtered_df.shape}")
    print(f"Filtered DataFrame {i+1} columns: {filtered_df.columns.tolist()}")
    print()

Filtered DataFrame 1 shape: (262079, 41)
Filtered DataFrame 1 columns: ['WLD/USDT_2024_1m', 'ICP/USDT_2024_1m', 'HBAR/USDT_2024_1m', 'INJ/USDT_2024_1m', 'IMX/USDT_2024_1m', 'FLOW/USDT_2024_1m', 'OP/USDT_2024_1m', 'FLOKI/USDT_2024_1m', 'SEI/USDT_2024_1m', 'IOTA/USDT_2024_1m', 'ARB/USDT_2024_1m', 'ATOM/USDT_2024_1m', 'ENS/USDT_2024_1m', 'AVAX/USDT_2024_1m', 'JASMY/USDT_2024_1m', 'GALA/USDT_2024_1m', 'JTO/USDT_2024_1m', 'VET/USDT_2024_1m', 'LINK/USDT_2024_1m', 'FET/USDT_2024_1m', 'EOS/USDT_2024_1m', 'SAND/USDT_2024_1m', 'FIL/USDT_2024_1m', 'CRV/USDT_2024_1m', 'DOT/USDT_2024_1m', 'AXS/USDT_2024_1m', 'EGLD/USDT_2024_1m', 'MANA/USDT_2024_1m', 'USDC/USDT_2024_1m', 'THETA/USDT_2024_1m', 'NEAR/USDT_2024_1m', 'LDO/USDT_2024_1m', 'OM/USDT_2024_1m', 'GRT/USDT_2024_1m', 'BONK/USDT_2024_1m', 'SUI/USDT_2024_1m', 'APT/USDT_2024_1m', 'TIA/USDT_2024_1m', 'NEO/USDT_2024_1m', 'ALGO/USDT_2024_1m', 'UNI/USDT_2024_1m']

Filtered DataFrame 2 shape: (262079, 25)
Filtered DataFrame 2 columns: ['WBTC/USDT_2024_1

In [34]:
filtered_dfs[1]

,WBTC/USDT_2024_1m,WBETH/USDT_2024_1m,PAXG/USDT_2024_1m,SOL/USDT_2024_1m,DEXE/USDT_2024_1m,DOGE/USDT_2024_1m,XRP/USDT_2024_1m,BTC/USDT_2024_1m,LTC/USDT_2024_1m,TRX/USDT_2024_1m,...,BCH/USDT_2024_1m,ETH/USDT_2024_1m,XTZ/USDT_2024_1m,ETC/USDT_2024_1m,AAVE/USDT_2024_1m,BNB/USDT_2024_1m,QNT/USDT_2024_1m,MKR/USDT_2024_1m,STX/USDT_2024_1m,ADA/USDT_2024_1m
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:01:00,0.490085,0.031666,0.747647,1.077888,-0.245788,0.346306,0.631702,0.664947,0.759945,1.328410,...,-0.275694,0.881157,0.743173,-0.000219,2.247302,1.070897,0.560231,0.458216,-0.258782,0.981832
2024-01-01 00:02:00,-0.001696,-0.001582,-0.000759,1.681351,-0.001959,-0.139593,0.631293,0.169447,-0.000117,-1.774474,...,3.559897,0.213826,0.000685,-0.000219,-0.135673,1.427147,0.559826,-0.920293,-0.074171,0.141296
2024-01-01 00:03:00,0.568023,-0.001582,-0.000759,0.939337,0.241871,0.831625,0.315969,1.321669,0.379679,0.220388,...,-0.547829,0.990526,0.000685,0.684416,1.766213,0.354507,-0.556436,0.917900,-0.998582,1.679236
2024-01-01 00:04:00,1.202768,-0.001582,-0.000759,0.267462,2.065489,0.345726,0.473286,0.908131,0.759007,0.442042,...,-1.369708,0.734374,0.742439,0.341865,0.407234,0.711331,0.559826,1.833755,1.588719,1.536445
2024-01-01 00:05:00,-0.304477,3.698485,-0.000759,0.066131,0.483178,0.137768,0.473056,0.370408,-0.000117,0.885164,...,1.367437,0.858971,0.000685,0.341709,0.135904,0.354165,0.001695,0.456868,0.221065,-2.092741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30 23:55:00,-0.327681,-0.001582,0.656018,0.092560,-0.042854,-0.150526,-0.202866,-0.001987,-0.244970,-0.192963,...,-0.181657,0.101372,0.000685,-0.000219,0.381655,-0.194469,0.001695,0.306437,0.960514,-0.210711
2024-06-30 23:56:00,-1.001454,-0.315899,-0.000759,-0.515459,-0.042858,-0.750046,-0.202909,-1.210917,-0.612534,-0.001327,...,-0.542976,-0.580955,0.000685,-0.317337,0.990885,-0.386378,2.000095,0.306316,-0.640650,-0.422901
2024-06-30 23:57:00,0.469226,-0.015133,-0.000759,0.045831,0.038940,0.199112,0.204811,0.336633,-0.000117,-0.001327,...,0.360137,0.104744,0.000685,-0.000219,-0.532906,-0.002658,0.998952,0.920614,-0.320751,0.001317


In [41]:

correlation_matrix,T,N,company_names = create_correlation_matrix(filtered_dfs[1]) 
C_g = calculate_C_g(correlation_matrix,T,N) #Modularity matrix
print(correlation_matrix.shape)
correlation_matrix_np = np.array(correlation_matrix)
louvain_communities = modified_louvain(C_g,correlation_matrix_np)

print("Global modularity of modified Louvain method: ", calculate_global_modularity(louvain_communities, C_g, correlation_matrix_np))


(25, 25)
Detected 2 Initially
Final number of communities: 2
Global modularity of modified Louvain method:  0.030636507532016695


In [42]:
louvain_company_communities = map_communities_to_company_names(louvain_communities,company_names)

In [43]:

print("LOUVAIN COMPANY COMMUNITIES 2")
print("---------------------------")
for louvain_company_community in louvain_company_communities:
    print(louvain_company_community)

LOUVAIN COMPANY COMMUNITIES 2
---------------------------
['PEPE/USDT_2024_1m', 'DOGE/USDT_2024_1m', 'XTZ/USDT_2024_1m', 'STX/USDT_2024_1m', 'BCH/USDT_2024_1m', 'MKR/USDT_2024_1m', 'XLM/USDT_2024_1m', 'SHIB/USDT_2024_1m', 'XRP/USDT_2024_1m', 'AAVE/USDT_2024_1m', 'LTC/USDT_2024_1m', 'SOL/USDT_2024_1m', 'ADA/USDT_2024_1m', 'ETC/USDT_2024_1m']
['NEXO/USDT_2024_1m', 'QNT/USDT_2024_1m', 'ETH/USDT_2024_1m', 'DEXE/USDT_2024_1m', 'WBETH/USDT_2024_1m', 'BTC/USDT_2024_1m', 'WBTC/USDT_2024_1m', 'FDUSD/USDT_2024_1m', 'BNB/USDT_2024_1m', 'TRX/USDT_2024_1m', 'PAXG/USDT_2024_1m']


# Round 3 of community detection 
---

In [44]:
# Next round of community detection 

#Create filtered dataframes

filtered_dfs = []
for i, louvain_company_community in enumerate(louvain_company_communities):
    filtered_df = merged_df_standardized_log_returns[louvain_company_community]
    filtered_dfs.append(filtered_df)
    print(f"Filtered DataFrame {i+1} shape: {filtered_df.shape}")
    print(f"Filtered DataFrame {i+1} columns: {filtered_df.columns.tolist()}")
    print()


Filtered DataFrame 1 shape: (262079, 14)
Filtered DataFrame 1 columns: ['PEPE/USDT_2024_1m', 'DOGE/USDT_2024_1m', 'XTZ/USDT_2024_1m', 'STX/USDT_2024_1m', 'BCH/USDT_2024_1m', 'MKR/USDT_2024_1m', 'XLM/USDT_2024_1m', 'SHIB/USDT_2024_1m', 'XRP/USDT_2024_1m', 'AAVE/USDT_2024_1m', 'LTC/USDT_2024_1m', 'SOL/USDT_2024_1m', 'ADA/USDT_2024_1m', 'ETC/USDT_2024_1m']

Filtered DataFrame 2 shape: (262079, 11)
Filtered DataFrame 2 columns: ['NEXO/USDT_2024_1m', 'QNT/USDT_2024_1m', 'ETH/USDT_2024_1m', 'DEXE/USDT_2024_1m', 'WBETH/USDT_2024_1m', 'BTC/USDT_2024_1m', 'WBTC/USDT_2024_1m', 'FDUSD/USDT_2024_1m', 'BNB/USDT_2024_1m', 'TRX/USDT_2024_1m', 'PAXG/USDT_2024_1m']



In [50]:
correlation_matrix,T,N,company_names = create_correlation_matrix(filtered_dfs[0]) 
C_g = calculate_C_g(correlation_matrix,T,N) #Modularity matrix
print(correlation_matrix.shape)
correlation_matrix_np = np.array(correlation_matrix)
louvain_communities = modified_louvain(C_g,correlation_matrix_np)

print("Global modularity of modified Louvain method: ", calculate_global_modularity(louvain_communities, C_g, correlation_matrix_np))


(14, 14)
Detected 14 Initially
Final number of communities: 14
Global modularity of modified Louvain method:  0.0


# Conclusion 
---

Conclusion:

The final communities you've detected make logical sense based on the types of cryptocurrencies and their respective market behaviors:

    Community 1 (speculative or meme-driven): These are smaller, more volatile cryptocurrencies that tend to react similarly to social media trends and investor sentiment.

    Community 2 (blue-chip or large-cap): These include more well-established cryptocurrencies that behave according to broader market forces, institutional investment, and more stable growth patterns.

At the third level of community detection, the algorithm splitting coins into individual communities suggests that you've already reached a point where further granularity does not offer significant new insights. The structure is already well-optimized and split in a way that reflects the natural market dynamics of these assets.

Thus, applying community detection further (beyond the second level) doesn't yield meaningful results because you've already found meaningful divisions between speculative assets and blue-chip assets. The results are sensible and reflect the market dynamics well.

The coins left in Community 1 after the first level of the Louvain method (which you applied to the entire network of 64 cryptocurrencies) appear to consist of several mid-to-small-cap cryptocurrencies. These coins are generally more speculative, have lower market caps, and may be influenced more by short-term market sentiment rather than fundamentals.
Analysis of Community 1 (The Remaining Cryptocurrencies):

Let’s break down the characteristics of the cryptocurrencies in Community 1 to understand the market dynamics at play.

    List of Coins:

        WLD/USDT_2024_1m, ICP/USDT_2024_1m, HBAR/USDT_2024_1m, INJ/USDT_2024_1m, IMX/USDT_2024_1m, FLOW/USDT_2024_1m, OP/USDT_2024_1m, FLOKI/USDT_2024_1m, SEI/USDT_2024_1m, IOTA/USDT_2024_1m, ARB/USDT_2024_1m, ATOM/USDT_2024_1m, ENS/USDT_2024_1m, AVAX/USDT_2024_1m, JASMY/USDT_2024_1m, GALA/USDT_2024_1m, JTO/USDT_2024_1m, VET/USDT_2024_1m, LINK/USDT_2024_1m, FET/USDT_2024_1m, EOS/USDT_2024_1m, SAND/USDT_2024_1m, FIL/USDT_2024_1m, CRV/USDT_2024_1m, DOT/USDT_2024_1m, AXS/USDT_2024_1m, EGLD/USDT_2024_1m, MANA/USDT_2024_1m, USDC/USDT_2024_1m, THETA/USDT_2024_1m, NEAR/USDT_2024_1m, LDO/USDT_2024_1m, OM/USDT_2024_1m, GRT/USDT_2024_1m, BONK/USDT_2024_1m, SUI/USDT_2024_1m, APT/USDT_2024_1m, TIA/USDT_2024_1m, NEO/USDT_2024_1m, ALGO/USDT_2024_1m, UNI/USDT_2024_1m.

Types of Cryptocurrencies in This Group:

    Mid-to-Small Cap Coins: Many of these coins, like WLD, ICP, IMX, ARB, FLOKI, JASMY, GALA, FIL, CRV, DOT, SAND, and AXS, are mid-to-small-cap assets. These coins tend to be more speculative and are often more volatile than larger, well-established coins like BTC, ETH, and BNB.

    Niche & High Volatility: Some of these cryptocurrencies, like FLOKI, FET, SAND, BONK, and GALA, are meme coins or projects with niche use cases. These coins can be heavily influenced by social media sentiment, speculative trading, or market trends, making them more likely to exhibit similar short-term price movements.

    Governance and Ecosystem Coins: Coins like ATOM (Cosmos), AVAX (Avalanche), NEAR, ALGO (Algorand), and UNI (Uniswap) are governance tokens and ecosystem coins. These projects have solid use cases and are often associated with blockchain ecosystems. Despite being mid-cap, they tend to show price movements driven by technology updates and ecosystem growth.

    Stablecoins: USDC, NEO, and LDO are also included in this community. USDC is a stablecoin, and NEO and LDO are stable, established projects that might provide some stabilization to the community's overall behavior.

    DeFi and NFT Projects: Some coins, like IMX (Immutable X), FIL (Filecoin), and GRT (The Graph), are tied to decentralized finance (DeFi) or NFT infrastructure projects, and could therefore exhibit a more shared correlation in terms of adoption, technological progress, and decentralized applications.

Why These Cryptocurrencies Formed a Single Community:

The reason why these cryptocurrencies were grouped into Community 1 is likely due to similar price movements or correlation of returns. Even though they may come from different sectors (meme coins, governance tokens, DeFi projects, etc.), their price movements over time could be sufficiently similar to cluster them together.

    High Short-Term Volatility: Many of the coins in this group have high volatility and are driven by short-term market sentiment (e.g., DOGE, SHIB, PEPE), leading them to move in similar ways when the broader market trends upward or downward.

    Speculative Behavior: The smaller-cap cryptocurrencies often exhibit speculative price movements. If there’s a surge in interest in the cryptocurrency market, many of these coins can show parallel price movement patterns.

    Shared Market Influences: Cryptocurrencies, even if they serve different use cases (meme coins vs. DeFi projects), can be influenced by similar macro factors, like the overall sentiment in the crypto market, regulatory news, or Bitcoin's performance. These shared influences can cause their prices to move in tandem, forming a community.

Why Community 2 Split Into Smaller Groups:

When you applied the algorithm to Community 2 (which had more established large-cap coins like BTC, ETH, and BNB), it split into two further sub-communities, which makes sense:

    Speculative vs. Established: Some of the coins in Community 2 (like PEPE, DOGE, and XRP) share more speculative, meme-driven behavior, while others (like BTC, ETH, BNB) are more fundamentally driven.

    Coin Types: The division of Community 2 into two groups likely reflects a separation between meme-driven/speculative coins (such as DOGE and PEPE) and more fundamentally-backed large-cap coins (such as BTC, ETH, and BNB).

This further split within Community 2 highlights how different factors (such as market sentiment vs. fundamentals) can create sub-groups even among larger, more established cryptocurrencies.
Conclusion:

    Community 1 consists of mid-to-small-cap cryptocurrencies that are more speculative, sentiment-driven, and volatile. These coins likely show similar short-term price movements due to social media influence, speculation, or similar market events. The fact that applying the algorithm further resulted in single-coin communities suggests that these coins' price movements are already very independent or idiosyncratic, which means that further splitting didn’t provide meaningful improvements.

    Community 2, on the other hand, contains larger, well-established cryptocurrencies that were split further into two sub-communities based on speculative behavior (meme coins) versus fundamentals (blue-chip coins). This shows that even well-established coins can be grouped based on different market dynamics (speculative versus fundamentally-backed).

The final communities you’ve identified make sense in terms of the types of coins:

    Community 1: A group of speculative and volatile cryptocurrencies.

    Community 2: A division between speculative assets (like meme coins) and blue-chip coins (like BTC, ETH).

This interpretation aligns with the behavior of cryptocurrencies in the market and how they are often influenced by different factors (sentiment vs. fundamentals).